In [ ]:
# Standard libraries
import os
import random
import numpy as np
from pathlib import Path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg

# TensorFlow / Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import (
    DenseNet169, InceptionV3, Xception, MobileNetV2, ResNet152, EfficientNetB4
)
from tensorflow.keras.layers import (
    Flatten, Dense, Dropout, Conv2D,
    GlobalAveragePooling2D, GlobalMaxPooling2D,
    Multiply, Add, Reshape, Activation
)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

# Evaluation
from sklearn.metrics import (
    classification_report, confusion_matrix
)

In [ ]:
train_dir = '/kaggle/input/skin-cancer-malignant-vs-benign/train' # Path to training data
validation_dir = '/kaggle/input/skin-cancer-malignant-vs-benign/test' # Path to validation data

classes = os.listdir(train_dir)
image_paths = [] 

for cls in classes:
    class_dir = os.path.join(train_dir, cls)
    images = os.listdir(class_dir)
    for img in images:
        image_paths.append((os.path.join(class_dir, img), cls))

# Randomly select 16 images from the dataset and display them
random_images = random.sample(image_paths, 16)
fig, axes = plt.subplots(4, 4, figsize=(10, 10))

for i, ax in enumerate(axes.flat):
    img_path, class_name = random_images[i]
    img = mpimg.imread(img_path)
    ax.imshow(img)
    ax.axis('off') 
    ax.set_title(class_name)

plt.show()

In [ ]:
#Display the number of images per class in the training data
classes = [cls.name for cls in train_dir.iterdir() if cls.is_dir()]
image_count = {cls: len(list((train_dir / cls).iterdir())) for cls in classes}
image_count = dict(sorted(image_count.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(12, 6))
bars = plt.bar(image_count.keys(), image_count.values(), color='skyblue')

for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.5,
             f'{int(bar.get_height())}', ha='center', va='bottom', color='black')

plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Number of Images per Class in the Training Data')
plt.xticks(rotation=90, ha='right')  
plt.tight_layout() 

plt.show()

In [ ]:
#Display the number of images per class in the test data
classes = [cls.name for cls in validation_dir.iterdir() if cls.is_dir()]
image_count = {cls: len(list((validation_dir / cls).iterdir())) for cls in classes}
image_count = dict(sorted(image_count.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(12, 6))
bars = plt.bar(image_count.keys(), image_count.values(), color='orange')

for bar in bars:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() - 0.5,  
             f'{int(bar.get_height())}', ha='center', va='bottom', color='black')

plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Number of Images per Class in the Test Data')
plt.xticks(rotation=90, ha='right') 
plt.tight_layout()  

plt.show()

In [ ]:
# CBAM Attention Block
def cbam_block(input_tensor, reduction_ratio=16):
    # Channel Attention
    channel = GlobalAveragePooling2D()(input_tensor)
    channel = Dense(input_tensor.shape[-1] // reduction_ratio, activation='relu')(channel)
    channel = Dense(input_tensor.shape[-1], activation='sigmoid')(channel)
    channel = Reshape((1, 1, input_tensor.shape[-1]))(channel)
    channel_attention = Multiply()([input_tensor, channel])
    
    # Spatial Attention
    spatial = Conv2D(1, kernel_size=(7, 7), padding='same', activation='sigmoid')(channel_attention)
    spatial_attention = Multiply()([channel_attention, spatial])
    
    return spatial_attention


In [ ]:
#Set image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

num_classes = len(train_generator.class_indices)
test_labels = validation_generator.classes
target_names = os.listdir(train_dir)

In [ ]:
# Create models with different architectures
def create_model(base_model_class, input_shape=(224, 224, 3)):
    base_model = base_model_class(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False 
    
    # Apply CBAM
    attention_output = cbam_block(base_model.output)
    
    x = Flatten()(attention_output)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=RMSprop(learning_rate=2e-5), loss="categorical_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
# List of models to try
models = [
    ('DenseNet169', DenseNet169),
    ('InceptionV3', InceptionV3),
    ('Xception', Xception),
    ('MobileNetV2', MobileNetV2),
    ('ResNet152', ResNet152),
    ('EfficientNetB4', EfficientNetB4)
]

best_model_name = None
best_val_accuracy = 0

# Train and evaluate each model
for model_name, model_class in models:
    print(f"Training {model_name}...")
    
    model = create_model(model_class)
    
    # Define checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        filepath=f'{model_name}.keras',
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    )
    
    # Train the model
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        verbose=1,
        callbacks=[checkpoint_callback]
    )
    
    val_accuracy = max(history.history['val_accuracy'])

    # Find the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_name = model_name

    print(f"{model_name} validation accuracy: {val_accuracy}")

print(f"The best performing model is {best_model_name} with a validation accuracy of {best_val_accuracy}")

In [ ]:
#Fuzzy Rank-based Ensemble:

# Function to get scores from the model
def getScore(model,test_imgs):
  res = model.predict(test_imgs)
  return res 

# Generate rank based on scores using a exponential function
def generateRank1(score,class_no):
  rank = np.zeros([class_no,1])
  scores = np.zeros([class_no,1])
  scores = score
  for i in range(class_no):
      rank[i] = 1 - np.exp(-((scores[i]-1)**2)/2.0)
  return rank

# Generate rank based on scores using a tanh function
def generateRank2(score,class_no):
  rank = np.zeros([class_no,1])
  scores = np.zeros([class_no,1])
  scores = score
  for i in range(class_no):
      rank[i] = 1 - np.tanh(((scores[i]-1)**2)/2)
  return rank

# Function to perform fusion of the results from three models
def doFusion(res1,res2,res3,label,class_no):
  cnt = 0
  id = []
  for i in range(len(res1)):
      rank1 = generateRank1(res1[i],class_no)*generateRank2(res1[i],class_no)
      rank2 = generateRank1(res2[i],class_no)*generateRank2(res2[i],class_no)
      rank3 = generateRank1(res3[i],class_no)*generateRank2(res3[i],class_no)
      rankSum = rank1 + rank2 + rank3
      rankSum = np.array(rankSum)
      scoreSum = 1 - (res1[i] + res2[i] + res3[i])/3
      scoreSum = np.array(scoreSum)
      
      fusedScore = (rankSum.T)*scoreSum
      cls = np.argmin(rankSum)
      id.append(cls)
  return id

In [ ]:
# Function that trains and saves the model
def train_and_save_model(model_name, base_model_class, checkpoint_path):
    print(f"\nTraining {model_name}...\n")
    model = create_model(base_model_class)
    
    # Define checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        monitor="val_accuracy",
        save_best_only=True,
        mode="max",
        verbose=1
    )
    
    # Train and validate the model
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=25,
        verbose=1,
        callbacks=[checkpoint_callback]
    )
    
    return history

In [ ]:
DenseNet169_history = train_and_save_model("DenseNet169", DenseNet169, "DenseNet169_best.keras")
InceptionV3_history = train_and_save_model("InceptionV3", InceptionV3, "InceptionV3_best.keras")
MobileNetV2_history = train_and_save_model("MobileNetV2", MobileNetV2, "MobileNetV2_best.keras")

DenseNet169_path = '/kaggle/working/DenseNet169_best.keras'
InceptionV3_path = '/kaggle/working/InceptionV3_best.keras'
MobileNetV2_path = '/kaggle/working/MobileNetV2_best.keras'

# Load the saved models
model0 = load_model(DenseNet169_path)
model1 = load_model(InceptionV3_path)
model2 = load_model(MobileNetV2_path)

In [ ]:
# Plot training history for each model
def plot_training_history(history):
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

print("DenseNet169 Training History: ")
plot_training_history(DenseNet169_history)

print("InceptionV3 Training History: ")
plot_training_history(InceptionV3_history)

print("MobileNetV2 Training History: ")
plot_training_history(MobileNetV2_history)

In [ ]:
print("BASE LEARNERS ACCURACY: ")
model0.evaluate(validation_generator)
model1.evaluate(validation_generator)
model2.evaluate(validation_generator)

# Predict and evaluate the models
res1 = model0.predict(validation_generator)
res2 = model1.predict(validation_generator) 
res3 = model2.predict(validation_generator)

# Perform ensemble fusion
predictedClass = doFusion(res1,res2,res3,test_labels,class_no=num_classes)

leb1 = np.argmax(res1,axis=-1)
leb2 = np.argmax(res2,axis=-1)
leb3 = np.argmax(res3,axis=-1)

print('Densenet-169 base learner:')
print(classification_report(test_labels, leb1,target_names = target_names,digits=4))
print('Inception base learner:')
print(classification_report(test_labels, leb2,target_names = target_names,digits=4))
print('MobileNetV2 base learner:')
print(classification_report(test_labels, leb3,target_names = target_names,digits=4))

print('Ensembled:')
print(classification_report(test_labels, predictedClass,target_names = target_names,digits=4))

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

print("Confusion Matrix of DenseNet169: ")
cm = confusion_matrix(test_labels, leb1)
class_names = list(validation_generator.class_indices.keys())
plot_confusion_matrix(cm, class_names)

print("Confusion Matrix of InceptionV3: ")
cm = confusion_matrix(test_labels, leb2)
class_names = list(validation_generator.class_indices.keys())
plot_confusion_matrix(cm, class_names)

print("Confusion Matrix of MobileNetV2: ")
cm = confusion_matrix(test_labels, leb3)
class_names = list(validation_generator.class_indices.keys())
plot_confusion_matrix(cm, class_names)

print("Confusion Matrix of Ensembled: ")
cm = confusion_matrix(test_labels, predictedClass)
class_names = list(validation_generator.class_indices.keys())
plot_confusion_matrix(cm, class_names)